<a href="https://colab.research.google.com/github/nikita383/Sculpture-Shipping/blob/main/Sculpture__shipping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import** **Libraries and Data**

In [ ]:
#import required libraries
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
# Dataset is now stored in a Pandas Dataframe

**Pre-processing data**

In [ ]:
#remove unused columns of data
train_x = train.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location', 'Cost'], axis = 1)
test_x = test.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location'], axis = 1)

train_y = train['Cost']

In [ ]:
#Deal with missing data

#categorical
train_x['Material'].fillna(train_x['Material'].mode()[0], inplace=True)
train_x['International'].fillna(train_x['International'].mode()[0], inplace=True)
train_x['Express Shipment'].fillna(train_x['Express Shipment'].mode()[0], inplace=True)
train_x['Installation Included'].fillna(train_x['Installation Included'].mode()[0], inplace=True)
train_x['Transport'].fillna(train_x['Transport'].mode()[0], inplace=True)
train_x['Fragile'].fillna(train_x['Fragile'].mode()[0], inplace=True)
train_x['Customer Information'].fillna(train_x['Customer Information'].mode()[0], inplace=True)
train_x['Remote Location'].fillna(train_x['Remote Location'].mode()[0], inplace=True)


test_x['Material'].fillna(test_x['Material'].mode()[0], inplace=True)
test_x['International'].fillna(test_x['International'].mode()[0], inplace=True)
test_x['Express Shipment'].fillna(test_x['Express Shipment'].mode()[0], inplace=True)
test_x['Installation Included'].fillna(test_x['Installation Included'].mode()[0], inplace=True)
test_x['Transport'].fillna(test_x['Transport'].mode()[0], inplace=True)
test_x['Fragile'].fillna(test_x['Fragile'].mode()[0], inplace=True)
test_x['Customer Information'].fillna(test_x['Customer Information'].mode()[0], inplace=True)
test_x['Remote Location'].fillna(test_x['Remote Location'].mode()[0], inplace=True)

In [ ]:
#label encode categorical data
cat_vars = ['Material', 'International', 'Express Shipment', 'Installation Included', 'Transport', 'Fragile', 'Customer Information', 'Remote Location']

from sklearn.preprocessing import LabelEncoder

label_train_x = train_x.copy()
label_test_x = test_x.copy()

label_encoder = LabelEncoder()

for var in cat_vars:
    label_train_x[var] = label_encoder.fit_transform(train_x[var])
    label_test_x[var] = label_encoder.fit_transform(test_x[var])

In [ ]:
#fill in NaN

label_train_x.fillna(0, inplace=True)
label_test_x.fillna(0, inplace=True)


In [ ]:
#demonstrate data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

normalized_train_x = label_train_x.copy()
normalized_test_x = label_test_x.copy()

# create scaler
scaler = MinMaxScaler()

num_vars = ['Artist Reputation', 'Height', 'Width', 'Weight', 'Price Of Sculpture', 'Base Shipping Price']


# apply transform
normalized_train_x[num_vars] = scaler.fit_transform(label_train_x[num_vars])
normalized_test_x[num_vars] = scaler.transform(label_test_x[num_vars])

**Model**

In [ ]:
##############################################             MODEL                #########################################################
import tensorflow as tf
tf.random.set_seed(123)
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, Flatten
from keras.regularizers import l1_l2
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping



#sample_size = x_train.shape[0] # number of samples in train set
#time_steps  = x_train.shape[1] # number of features in train set
#input_dimension = 1               # each feature is represented by 1 number

#x_train_reshaped = x_train.reshape(sample_size,time_steps,input_dimension)


early_stop = EarlyStopping(monitor='val_mean_absolute_percentage_error', min_delta=0, patience=4, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

model = Sequential()

#model.add(Conv1D(filters =32, kernel_size = 3, input_shape = (14, 1), padding='valid'))
model.add(Dense(32, input_shape = (None, 14), kernel_regularizer=l1_l2(l1 = 1e-5, l2 = 1e-5), bias_regularizer=l1_l2(l1 = 1e-5, l2 = 1e-5), activity_regularizer=l1_l2(l1 = 1e-5, l2 = 1e-5)))
model.add(Activation('relu'))

model.add(Dropout( 1e-4))

#model.add(Conv1D(filters =64, kernel_size = 5, padding='valid'))
model.add(Dense(64, kernel_regularizer=l1_l2(l1 = 1e-5, l2 = 1e-5), bias_regularizer=l1_l2(l1 = 1e-5, l2 = 1e-5), activity_regularizer=l1_l2(l1 = 1e-5, l2 = 1e-5)))
model.add(Activation('relu'))

#model.add(Flatten())
model.add(Dropout( 1e-4))

model.add(Dense( 1))
model.add(Activation('relu'))

model.summary()

model.compile(loss='mean_absolute_percentage_error', optimizer='adam', metrics=['mean_absolute_percentage_error', 'accuracy'])

history = model.fit(normalized_train_x, train_y, batch_size=32, epochs=50, validation_split=0.2, callbacks=[early_stop], verbose = 1)

fig, axes = plt.subplots(2,1)
axes[0].plot(history.history['accuracy'])
axes[0].set_title('Model Accuracy')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Epoch')

fig.subplots_adjust(hspace=0.5)

axes[1].plot(history.history['mean_absolute_percentage_error'], 'g')
axes[1].plot(history.history['val_mean_absolute_percentage_error'], 'b')
axes[1].set_title('Model Loss')
axes[1].set_ylabel('MAE Loss')
axes[1].set_xlabel('Epoch')

In [ ]:
y_prediction = model.predict(normalized_test_x)

**Create Output**

In [ ]:
ids = test['Customer Id'].values

In [ ]:
prediction = pd.DataFrame()
prediction['Customer Id'] = test['Customer Id']
prediction['Cost'] = y_prediction

In [ ]:
prediction.to_csv("foo(3).csv", index = False)